# Fine-Tuning Gemma-3 4B parameters with Unsloth

## Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False,
    full_finetuning = False,
    #token = "", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model.language_model` require gradients


### Data Prep

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",   # important for <start_of_turn>user/model
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

SFT_PATH = "path"

dataset = load_dataset(
    "json",
    data_files = SFT_PATH,
    split = "train",
)

print(dataset)
print(dataset[0])    # quick sanity check: should show {"messages": [...]}

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 1609
})
{'messages': [{'role': 'system', 'content': 'شما دستیار ارشد صدور و نگهداشت بیمه آتش\u200cسوزی هستید. با توجه به risk_score و ویژگی\u200cهای مشتری و احتمال ریزش، تحلیل ریسک و برنامه نگهداشت مرحله\u200cبندی\u200cشده ارائه کنید و فقط JSON استاندارد برگردانید.'}, {'role': 'user', 'content': '{"customer_profile": {"years_with_company": null, "total_premium_paid": null, "renewal_count": null, "avg_claim_amount": null, "claim_count": null, "days_since_first_policy": null, "days_since_last_renewal": null, "days_to_expiration": null}, "underwriting": {"risk_score": 20, "risk_level": "پایین", "coverage": null}, "churn": {"p_churn": 0.0, "segment": "Low", "top_reasons": []}, "constraints": {"max_discount_pct": 10, "allowed_actions": ["یادآور", "تماس کارشناس", "تقسیط", "پیشنهاد پوشش افزوده", "بن باشگاه"]}}'}, {'role': 'assistant', 'content': '{"summary": "ریسک این مورد پایین ارزیابی شد. پیشنهاد پذیرش بدون شرط و با نرخ استاندارد است.", "r

In [ ]:
def formatting_prompts_func(examples):
    """
    examples["messages"] is a list of conversations.
    Each conversation is a list of {role, content} dicts.
    We turn each conversation into a single training string
    using Gemma-3's chat template.
    """
    convos = examples["messages"]          # list of conversations
    texts  = []

    for convo in convos:
        # convo is: [{"role": "system", "content": ...}, {"role":"user",...}, {"role":"assistant",...}]
        text = tokenizer.apply_chat_template(
            convo,
            tokenize             = False,
            add_generation_prompt = False,   # we include the assistant answer in training
        )
        # Gemma-3 template typically starts with "<bos>"
        if text.startswith("<bos>"):
            text = text[len("<bos>"):]
        texts.append(text)

    return { "text": texts }

In [ ]:
dataset = dataset.map(
    formatting_prompts_func,
    batched = True,
    remove_columns = dataset.column_names,   # optional: keep only "text" to save memory
)

print(dataset[0]["text"][:500])   # see how one training example looks

Map:   0%|          | 0/1609 [00:00<?, ? examples/s]

<start_of_turn>user
شما دستیار ارشد صدور و نگهداشت بیمه آتش‌سوزی هستید. با توجه به risk_score و ویژگی‌های مشتری و احتمال ریزش، تحلیل ریسک و برنامه نگهداشت مرحله‌بندی‌شده ارائه کنید و فقط JSON استاندارد برگردانید.

{"customer_profile": {"years_with_company": null, "total_premium_paid": null, "renewal_count": null, "avg_claim_amount": null, "claim_count": null, "days_since_first_policy": null, "days_since_last_renewal": null, "days_to_expiration": null}, "underwriting": {"risk_score": 20, "risk_le


### Train The Model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig
import torch
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 30,
        learning_rate = 5e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nشما دستیار ارشد صدور و نگهداشت بیمه آتش\u200cسوزی هستید. با توجه به risk_score و ویژگی\u200cهای مشتری و احتمال ریزش، تحلیل ریسک و برنامه نگهداشت مرحله\u200cبندی\u200cشده ارائه کنید و فقط JSON استاندارد برگردانید.\n\n{"customer_profile": {"years_with_company": null, "total_premium_paid": null, "renewal_count": null, "avg_claim_amount": null, "claim_count": null, "days_since_first_policy": null, "days_since_last_renewal": null, "days_to_expiration": null}, "underwriting": {"risk_score": 95, "risk_level": "خیلی بالا", "coverage": null}, "churn": {"p_churn": 0.0, "segment": "Low", "top_reasons": []}, "constraints": {"max_discount_pct": 10, "allowed_actions": ["یادآور", "تماس کارشناس", "تقسیط", "پیشنهاد پوشش افزوده", "بن باشگاه"]}}<end_of_turn>\n<start_of_turn>model\n{"summary": "این پرونده به دلیل درخواست پوشش خطر تبعی جنگ و بمباران، که طبق قوانین ایران غیرقابل ارائه است، رد می\u200cشود. این درخواست به دلیل مغایرت با مقررات، فاقد اعتبار است.", "retention_plan": [

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                                                                           {"summary": "این پرونده به دلیل درخواست پوشش خطر تبعی جنگ و بمباران، که طبق قوانین ایران غیرقابل ارائه است، رد می\u200cشود. این درخواست به دلیل مغایرت با مقررات، فاقد اعتبار است.", "retention_plan": [{"step": 1, "action": "غیرقابل محاسبه"}], "underwriting_notes": ["پذیرش پوشش خطر تبعی جنگ و بمباران در بیمه\u200cنامه\u200cهای آتش\u200cسوزی در ایران ممنوع بوده و این درخواست از نظر مقررات بیمه\u200cای نامعتبر است."], "cx_message_short": "این پرونده به دلیل درخواست پوشش خطر تبعی جنگ و بمباران، که طبق قوانین ایران غیرقابل ارائه است، رد می\u200cشود. این درخواست به دلیل مغایرت با مقررات، فاقد اعتبار است."}<end_of_turn>\n'

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.59 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,609 | Num Epochs = 2 | Total steps = 404
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248 of 4,314,980,720 (0.35% trained)


Step,Training Loss
1,1.048100
2,1.042300
3,1.013200
4,1.075000
5,1.130100
6,1.036900
7,1.089200
8,1.106000
9,1.311100
10,1.231600


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3457.9671 seconds used for training.
57.63 minutes used for training.
Peak reserved memory = 6.607 GB.
Peak reserved memory for training = 1.017 GB.
Peak reserved memory % of max memory = 44.821 %.
Peak reserved memory for training % of max memory = 6.899 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer
import torch
import json

# Re-attach the chat template to be safe
tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")

# === Test A: Fibonacci ===
# Gemma 3 is multimodal, so transformers expects content to be a list of dicts
messages = [
    {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant."}]},
    {"role": "user", "content": [{"type": "text", "text": "Continue the sequence: 1, 1, 2, 3, 5, 8,"}]}
]

# Apply the chat template (do not change the format)
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,  # Must add for generation
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)

# Generate output
outputs = model.generate(
    **inputs.to("cuda"),
    max_new_tokens=64,
    temperature=1.0,
    top_p=0.95,
    top_k=64,
)

print("Fibonacci test:")
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


# === Test B: Sky question (streamed) ===
messages = [
    {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant."}]},
    {"role": "user", "content": [{"type": "text", "text": "Why is the sky blue?"}]}
]

# Apply the chat template for the sky question
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)

# Generate streamed output
print("\nSky question (streamed):")
_ = model.generate(
    **inputs.to("cuda"),
    max_new_tokens=128,
    temperature=1.0,
    top_p=0.95,
    top_k=64,
    streamer=TextStreamer(tokenizer, skip_prompt=True),
)


Fibonacci test:
['user\nYou are a helpful assistant.\n\nContinue the sequence: 1, 1, 2, 3, 5, 8,\nmodel\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6']

Sky question (streamed):
That's a fantastic and common question! The sky is blue because of a phenomenon called **Rayleigh scattering**. Here’s a breakdown:

1. **Sunlight and Colors:** Sunlight, which looks white to us, is actually made up of all the colors of the rainbow.

2. **Scattering:** When sunlight enters the Earth's atmosphere, it bumps into tiny air molecules (mostly nitrogen and oxygen). This interaction causes the sunlight to scatter. 

3. **Rayleigh Scattering & Blue/Violet:** Rayleigh scattering is more effective at scattering *shorter* wavelengths of light (like blue and violet) than


In [ ]:
# ------------------------------------------------------------------------------
# 3) REAL USE-CASE: insurance + churn JSON (input_block)
#    NOTE: you must define `input_block` before this section, e.g.:
#
# input_block = {
#   "customer_profile": {...},
#   "underwriting": {...},
#   "churn": {...},
#   "constraints": {...}
# }
# ------------------------------------------------------------------------------

messages = [
    {
        "role": "system",
        "content": (
            "You are an advanced fire insurance underwriting and retention assistant. "
            "Given the JSON from the user, return ONLY one JSON object with the keys: "
            "summary, retention_plan, underwriting_notes, cx_message_short. "
            "Do NOT add any extra text before or after the JSON."
        ),
    },
    {
        "role": "user",
        # we trained with JSON as string in user content
        "content": json.dumps(input_block, ensure_ascii=False),
    },
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize              = True,
    return_tensors        = "pt",
    return_dict           = True,
)

outputs = model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 512,   # allow enough room for full JSON
    temperature    = 0.7,   # a bit more deterministic
    top_p          = 0.9,
    top_k          = 64,
)

result = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("\nRaw model output for insurance case:")
print(result)

# If you want to parse JSON from result:
# start = result.find("{")
# end   = result.rfind("}")
# parsed = json.loads(result[start:end+1])

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# ------------------------------------------------------------------------------
# 4) Save fine-tuned model locally
# ------------------------------------------------------------------------------
save_directory = "path"

# Change "gemma-3-finetune" to any folder name you like
model.save_pretrained(save_directory)     # Local saving
tokenizer.save_pretrained(save_directory)
print(f"Model and tokenizer saved to {save_directory}")

# Optional: push to Hugging Face Hub
# model.push_to_hub("HF_ACCOUNT/gemma-3-finetune", token="")
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3-finetune", token="")

Model and tokenizer saved to /content/drive/MyDrive/Thesis Project Final/Data-Process


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
# ------------------------------------------------------------------------------
# 5) (Optional) Reload fine-tuned model later with FastModel
# ------------------------------------------------------------------------------

if False:  # Change to True when you want to test reloading
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name     = "gemma-3-finetune",  # Folder you just saved
        max_seq_length = 2048,
        load_in_4bit   = True,
    )

    messages = [
        {
            "role": "user",
            "content": "What is Gemma-3?"
        }
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        tokenize              = True,
        return_tensors        = "pt",
        return_dict           = True,
    )

    print("\nReload test:")
    _ = model.generate(
        **inputs.to("cuda"),
        max_new_tokens = 64,
        temperature    = 1.0,
        top_p          = 0.95,
        top_k          = 64,
        streamer       = TextStreamer(tokenizer, skip_prompt=True),
    )

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp.

In [ ]:
# ------------------------------------------------------------------------------
# 6) Optional: merge LoRA and export (merged / GGUF)
# ------------------------------------------------------------------------------

if False:  # Change to True to save merged finetuned model (single full model)
    model.save_pretrained_merged("gemma-3-finetune-merged", tokenizer)

if False:  # Change to True to upload merged finetuned model
    model.push_to_hub_merged(
        "mahdishahsavari/gemma-3-finetune-merged",
        tokenizer,
        token = "",
    )

if True:  # Change to True to save GGUF for Ollama / llama.cpp
    model.save_pretrained_gguf(
        "gemma-3-finetune-gguf",
        tokenizer,
        quantization_method = "Q8_0",  # Supports Q8_0, BF16, F16
    )

if True:  # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "mahdishahsavari/gemma-3-finetune-gguf",
        tokenizer,
        quantization_method = "Q8_0",
        token = "",
    )

Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:27<00:00, 13.52s/it]


Unsloth: Merge process complete. Saved to `/content/gemma-3-finetune-gguf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['gemma-3-4b-it.F16.gguf', 'gemma-3-4b-it.F16-mmproj.gguf']
Unsloth: [2] Converting GGUF f16 into q8_0. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['gemma-3-4b-it.Q8_0.gguf', 'gemma-3-4b-it.F16-mmproj.gguf']


Unsloth: example usage 

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Uploading gemma-3-4b-it.F16-mmproj.gguf...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...a-3-4b-it.F16-mmproj.gguf:   1%|          | 4.57MB /  851MB            

Uploading config.json...
Uploading Ollama Modelfile...
Unsloth: Successfully uploaded GGUF to https://huggingface.co/mahdishahsavari/gemma-3-finetune-gguf
Unsloth: Cleaning up temporary files...
